# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [ ]:

#URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [ ]:
YOUR_CITY = ''

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib2
import json

import pandas as pd

import re
import requests

from sklearn.model_selection import cross_val_score
import patsy
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV



/Users/dnay/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/dnay/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

In [2]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 1000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

results = []

for city in ['New+York', 'Chicago', 'San+Francisco', 'Portland', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'San+Diego', 'Denver', 'Spokane', 'Tampa','Columbus','Las+Vegas']:
    for start in range(0, max_results_per_city, 10):
        url_ = 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l='+city+'&start='+str(start)
        soupy = (BeautifulSoup(urllib2.urlopen(url_).read()))
        results.append(soupy.find_all('div', class_=' row result'))

/Users/dnay/anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/dnay/anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it
    - Remember to use `try/except` if you anticipate errors
- **Test** the functions on the results above and simple examples

In [3]:
## YOUR CODE HERE
def get_location(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, span_loc in enumerate(div.find_all('span', class_='location')):
            di[id2] = span_loc.text
    return di

def get_company(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, span_comp in enumerate(div.find_all('span', class_='company')):
            di[id2] = span_comp.text
    return di
        
def get_jobtitle(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, header in enumerate(div.find_all('h2')):
            di[id2] = header.text
    return di

def get_salary(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, td_snip in enumerate(div.find_all('td', class_='snip')):
            for id3, nobr in enumerate(td_snip.find_all('nobr')):
                di[id2] = nobr.text
    return di

In [4]:
li = []
for i in results:
    li.append(get_location(i))
df = pd.DataFrame(li)

In [5]:
li = []
for i in results:
    li.append(get_jobtitle(i))
df['job'] = pd.DataFrame(li)

In [6]:
li = []
for i in results:
    li.append(get_salary(i))
df['salary'] = pd.DataFrame(li)

In [7]:
li = []
for i in results:
    li.append(get_company(i))
df['company'] = pd.DataFrame(li)


In [8]:
df.columns = ['city', 'job', 'salary', 'company']
df

,city,job,salary,company
0,"New York, NY",\nData Scientist\n,NaN,\n\n pulseData\n
1,"New York, NY 10011 (Chelsea area)",\nSenior Data Scientist (Product)\n,NaN,\n\n\n Spotify\n
2,"New York, NY","\nICG Client Analytics, Full Time Associate St...",NaN,\n\n\n Citi\n
3,"New York, NY",\nData Scientist\n,NaN,\n\n pulseData\n
4,"New York, NY",\nData Scientist\n,NaN,\n\n Dia&Co\n
5,"New York, NY",\nData Scientist\n,NaN,\n\n Astrid Solutions\n
6,"New York, NY",\nData Scientist\n,NaN,\n\n pulseData\n
7,"New York, NY 10111 (Midtown area)",\nSenior Data Scientist\n,NaN,\n\n\n Capital Group\n
8,"New York, NY",\nData Scientist\n,NaN,\n\n pulseData\n
9,"New York, NY",\nResearch Analyst - Data Analytics and Artifi...,NaN,\n\n Lux Research\n


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [ ]:
## YOUR CODE HERE
#did above

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [9]:
#drop nan's and remove duplicates
df.dropna(how='any', axis=0, inplace=True)
df.drop_duplicates(inplace=True)

df.head()

(122, 4)


,city,job,salary,company
13,"New York, NY",\nData Analyst - Cardiology\n,"$33,650 a year",\n\n\n Columbia University\n
16,"New York, NY",\nAssociate Research Scientist\n,"$120,000 a year",\n\n\n Columbia University\n
18,"New York, NY 10013 (Tribeca area)",\nResearch and Brand Strategy Apprentice\n,"$130,000 - $165,000 a year",\n\n Insight Strategy Group\n
22,"New York, NY",\nCity Research Scientist II\n,"$70,286 - $85,664 a year",\n\n\n City of New York\n
23,"New York, NY",\nBusiness Intelligence Engineer\n,"$150,000 a year",\n\n\n Amazon Corporate LLC\n


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [10]:
list1 = []
for i in df['city']:
    list1.append(str(i).split(',')[0])
df['city'] = list1
df.head()

,city,job,salary,company
13,New York,\nData Analyst - Cardiology\n,"$33,650 a year",\n\n\n Columbia University\n
16,New York,\nAssociate Research Scientist\n,"$120,000 a year",\n\n\n Columbia University\n
18,New York,\nResearch and Brand Strategy Apprentice\n,"$130,000 - $165,000 a year",\n\n Insight Strategy Group\n
22,New York,\nCity Research Scientist II\n,"$70,286 - $85,664 a year",\n\n\n City of New York\n
23,New York,\nBusiness Intelligence Engineer\n,"$150,000 a year",\n\n\n Amazon Corporate LLC\n


In [11]:
#had to deal with unicode in order to send to csv, with both company and job title
li = []
for i in df['job']:
    li.append(str(i.encode('ascii', 'ignore')).split())
di1 = {}
for id1, string in enumerate(li):
    for letters in string:
        #skip newlines
        if '\\n' in letters:
            continue
        elif id1 not in di1:
            di1[id1] = str(letters.encode('ascii', 'ignore'))
        else:
            di1[id1] += ' ' + str(letters)
li2 = []
for i in range(len(di1)):
    li2.append(di1[i])
df['job'] = li2


listyx = []
for i in df['company']:
    listyx.append(str(i).split())
dictyx = {}
for id1, string in enumerate(listyx):
    for letters in string:
        if '\\n' in letters:
            continue
        elif id1 not in dictyx:
            dictyx[id1] = str(letters)
        else:
            dictyx[id1] += ' ' + str(letters)
listyxx = []
for i in range(len(dictyx)):
    listyxx.append(dictyx[i])
df['company'] = listyxx
df.head()

,city,job,salary,company
13,New York,Data Analyst - Cardiology,"$33,650 a year",Columbia University
16,New York,Associate Research Scientist,"$120,000 a year",Columbia University
18,New York,Research and Brand Strategy Apprentice,"$130,000 - $165,000 a year",Insight Strategy Group
22,New York,City Research Scientist II,"$70,286 - $85,664 a year",City of New York
23,New York,Business Intelligence Engineer,"$150,000 a year",Amazon Corporate LLC


In [13]:
## YOUR CODE HERE
#salary column clean up
listyx = []
for i in df['salary']:
    listyx.append(str(i).replace('$', '').replace(',', '').replace('-', '').replace(' a year', '')
                .replace('a week', '').replace('a month', '').replace('an hour', ''))

listyx = [i.split() for i in listyx]
num_list = []
for id1, i in enumerate(listyx):
    for j in i:
        num_list += [(id1, float(j))]

        
dictyx = {}
for j, k in num_list:
    if j not in dictyx:
        dictyx[j] = k
    else:
        dictyx[j] += k
        dictyx[j] /= 2

for i in dictyx:
    if dictyx[i] < 200:
        dictyx[i] *= (52*40)
    elif dictyx[i] < 4000:
        dictyx[i] *= 52
    elif dictyx[i] < 10000:
        dictyx[i] *= 12
salaries = []
for i in range(len(dictyx)):
    salaries.append(dictyx[i])

In [15]:
df['salary'] = salaries
df.head()

,city,job,salary,company
13,New York,Data Analyst - Cardiology,33650.0,Columbia University
16,New York,Associate Research Scientist,120000.0,Columbia University
18,New York,Research and Brand Strategy Apprentice,147500.0,Insight Strategy Group
22,New York,City Research Scientist II,77975.0,City of New York
23,New York,Business Intelligence Engineer,150000.0,Amazon Corporate LLC


### Save your results as a CSV

In [16]:
df.to_csv('Indeed Job Data')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [ ]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [19]:
## YOUR CODE HERE
import numpy as np

#reasonable breaking points,
q1 = df['salary'].quantile(.25)
median1 = np.median(df['salary'])
q3 = df['salary'].quantile(.75)
your_highness = []
for i in df['salary']:
    if i < q1:
        your_highness.append('super_low')
    elif i < median1:
        your_highness.append('low')
    elif i < q3:
        your_highness.append('high')
    else:
        your_highness.append('super_high')
df['how high'] = your_highness
df.head()

listyx = []
for i in df['salary']:
    if i > median1:
        listyx.append(1)
    else:
        listyx.append(0)
df['is_high'] = listyx
df.head()

,city,job,salary,company,how high,is_high
13,New York,Data Analyst - Cardiology,33650.0,Columbia University,super_low,0
16,New York,Associate Research Scientist,120000.0,Columbia University,super_high,1
18,New York,Research and Brand Strategy Apprentice,147500.0,Insight Strategy Group,super_high,1
22,New York,City Research Scientist II,77975.0,City of New York,low,0
23,New York,Business Intelligence Engineer,150000.0,Amazon Corporate LLC,super_high,1


#### Thought experiment: What is the baseline accuracy for this model?

In [22]:
## YOUR CODE HERE
# Baseline is the most common occurence
print df['how high'].value_counts(), '\n'
print "Baseline: ",34. / (34+31+30+27)

super_high    34
super_low     31
low           30
high          27
Name: how high, dtype: int64 

Baseline:  0.27868852459


#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [23]:
## YOUR CODE HERE

import statsmodels.formula.api as sm
model = sm.logit(
    "is_high ~ city",
    data = df
).fit()

model.summary()

         Current function value: 0.399040
         Iterations: 35


/Users/dnay/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/dnay/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:971: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
/Users/dnay/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/dnay/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/dnay/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                is_high   No. Observations:                  122
Model:                          Logit   Df Residuals:                       86
Method:                           MLE   Df Model:                           35
Date:                Tue, 01 Nov 2016   Pseudo R-squ.:                  0.4243
Time:                        06:02:26   Log-Likelihood:                -48.683
converged:                      False   LL-Null:                       -84.564
                                        LLR p-value:                 0.0002461
============================================================================================
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                 -225.8252        nan        nan        nan           nan       nan
city[T.Atlanta]            224.9089        nan        nan        nan           nan       nan
city[T.Bellevue]           226.5183        nan        nan        nan           nan       nan
city[T.Boulder]            225.1320        nan        nan        nan           nan       nan
city[T.Buckley AFB]        203.0826        nan        nan        nan           nan       nan
city[T.Burbank]            241.0538        nan        nan        nan           nan       nan
city[T.Chatsworth]         203.0899        nan        nan        nan           nan       nan
city[T.Chicago]            225.8252        nan        nan        nan           nan       nan
city[T.Dallas]             201.6951        nan        nan        nan           nan       nan
city[T.Deerfield]          240.4703        nan        nan        nan           nan       nan
city[T.Denver]             225.8252        nan        nan        nan           nan       nan
city[T.El Segundo]         195.0707        nan        nan        nan           nan       nan
city[T.Evanston]           201.6959        nan        nan        nan           nan       nan
city[T.Irving]             225.8252        nan        nan        nan           nan       nan
city[T.Issaquah]           201.6964        nan        nan        nan           nan       nan
city[T.La Jolla]           204.0466        nan        nan        nan           nan       nan
city[T.Los Angeles]        195.0707        nan        nan        nan           nan       nan
city[T.New York]           226.9778        nan        nan        nan           nan       nan
city[T.Newtown]            240.4703        nan        nan        nan           nan       nan
city[T.Norcross]           240.4703        nan        nan        nan           nan       nan
city[T.Northbrook]         241.0614        nan        nan        nan           nan       nan
city[T.Pasadena]           225.8252        nan        nan        nan           nan       nan
city[T.Philadelphia]       225.8252        nan        nan        nan           nan       nan
city[T.Pittsburgh]         208.9617        nan        nan        nan           nan       nan
city[T.Plano]              225.8252        nan        nan        nan           nan       nan
city[T.Plymouth Meeting]   240.4642        nan        nan        nan           nan       nan
city[T.Portland]           209.0836        nan        nan        nan           nan       nan
city[T.Post Falls]         201.0457        nan        nan        nan           nan       nan
city[T.Roswell]            201.6959        nan        nan        nan           nan       nan
city[T.San Bruno]          240.4703        nan        nan        nan           nan       nan
city[T.San Diego]          226.5183        nan        nan        nan           nan       nan
city[T.San Francisco]      227.5299        nan        nan        nan           

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [24]:
def job_words(word):    
    temp = []
    for i in df['job']:
        if word in i.lower():
            temp.append(1)
        else:
            temp.append(0)
    df[word] = temp
def company_words(word):
    temp = []
    for i in df['company']:
        if word in i.lower():
            temp.append(1)
        else:
            temp.append(0)
    df[word] = temp
jobwords = ['manager', 'senior', 'research', 'machine', 'junior',
         'python', 'developer', 'analyst', 'scientist','data','associate']
#couldnt use company words for some reason, kept getting 'singular matrix' error down below
companywords = ['llc','bank','education','university','ibm','amazon','court']
for i in jobwords:
    job_words(i)
for i in companywords:
    company_words(i)

In [25]:
df

,city,job,salary,company,how high,is_high,manager,senior,research,machine,...,scientist,data,associate,llc,bank,education,university,ibm,amazon,court
13,New York,Data Analyst - Cardiology,33650.0,Columbia University,super_low,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
16,New York,Associate Research Scientist,120000.0,Columbia University,super_high,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
18,New York,Research and Brand Strategy Apprentice,147500.0,Insight Strategy Group,super_high,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
22,New York,City Research Scientist II,77975.0,City of New York,low,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
23,New York,Business Intelligence Engineer,150000.0,Amazon Corporate LLC,super_high,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
25,New York,Data Engineer,135000.0,Paperless Post,super_high,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
26,New York,Research Data Associate,120000.0,NYU Langone Medical Center,super_high,1,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
28,New York,"Senior Analyst, Quantitative Research",140000.0,Kelton Global,super_high,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
31,New York,"Research Engineer, Senior",175000.0,Yahoo! Inc.,super_high,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
34,New York,"Research Analyst - Robotics, AI and Machine Le...",82500.0,Lux Research,low,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [53]:
model = sm.logit(
    "is_high ~ city + developer + senior + associate + manager + machine",
    data = df
).fit()

model.summary()

         Current function value: 0.390972
         Iterations: 35


/Users/dnay/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                is_high   No. Observations:                  122
Model:                          Logit   Df Residuals:                       81
Method:                           MLE   Df Model:                           40
Date:                Tue, 01 Nov 2016   Pseudo R-squ.:                  0.4359
Time:                        06:09:56   Log-Likelihood:                -47.699
converged:                      False   LL-Null:                       -84.564
                                        LLR p-value:                 0.0009199
============================================================================================
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                 -227.1710        nan        nan        nan           nan       nan
city[T.Atlanta]            225.9422        nan        nan        nan           nan       nan
city[T.Bellevue]           227.6291        nan        nan        nan           nan       nan
city[T.Boulder]            226.0118        nan        nan        nan           nan       nan
city[T.Buckley AFB]        197.4714        nan        nan        nan           nan       nan
city[T.Burbank]            251.3096        nan        nan        nan           nan       nan
city[T.Chatsworth]         193.4168   1.61e+07    1.2e-05      1.000     -3.15e+07  3.15e+07
city[T.Chicago]            226.9997        nan        nan        nan           nan       nan
city[T.Dallas]             193.4040   1.63e+07   1.19e-05      1.000     -3.19e+07  3.19e+07
city[T.Deerfield]          244.2958        nan        nan        nan           nan       nan
city[T.Denver]             226.8892        nan        nan        nan           nan       nan
city[T.El Segundo]         196.7988        nan        nan        nan           nan       nan
city[T.Evanston]           192.9853   6.71e+06   2.88e-05      1.000     -1.31e+07  1.31e+07
city[T.Irving]             227.1710        nan        nan        nan           nan       nan
city[T.Issaquah]           193.4029   1.63e+07   1.19e-05      1.000     -3.19e+07  3.19e+07
city[T.La Jolla]           193.4029   1.63e+07   1.19e-05      1.000     -3.19e+07  3.19e+07
city[T.Los Angeles]        196.7988        nan        nan        nan           nan       nan
city[T.New York]           228.1766        nan        nan        nan           nan       nan
city[T.Newtown]            244.9431        nan        nan        nan           nan       nan
city[T.Norcross]           244.9431        nan        nan        nan           nan       nan
city[T.Northbrook]         251.5322        nan        nan        nan           nan       nan
city[T.Pasadena]           226.4228        nan        nan        nan           nan       nan
city[T.Philadelphia]       227.1710        nan        nan        nan           nan       nan
city[T.Pittsburgh]         205.7821        nan        nan        nan           nan       nan
city[T.Plano]              227.0315        nan        nan        nan           nan       nan
city[T.Plymouth Meeting]   244.9427        nan        nan        nan           nan       nan
city[T.Portland]           211.2117        nan        nan        nan           nan       nan
city[T.Post Falls]         193.4029   1.63e+07   1.19e-05      1.000     -3.19e+07  3.19e+07
city[T.Roswell]            193.4029   1.63e+07   1.19e-05      1.000     -3.19e+07  3.19e+07
city[T.San Bruno]          244.9422        nan        nan        nan           nan       nan
city[T.San Diego]          227.5566        nan        nan        nan           nan       nan
city[T.San Francisco]      228.8119        nan        nan        nan           

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [30]:
## YOUR CODE HERE
## YOUR CODE HERE

X = patsy.dmatrix('~ C(city) + C(manager) + C(senior) + C(junior) +C(research) + C(associate) \
                  + C(machine) + C(python) + C(developer) + C(analyst) + C(scientist)', df)
y = df['how high']

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [31]:
## YOUR CODE HERE
logreg = LogisticRegression()
C_vals = [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty':penalties, 'C':C_vals}, verbose=True, cv=5, scoring='f1_macro')
gs.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/dnay/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    1.1s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1_macro',
       verbose=True)

In [32]:
gs.best_params_

{'C': 0.75, 'penalty': 'l1'}

In [33]:
logreg = LogisticRegression(C=0.75, penalty='l1', solver='liblinear')
logreg.fit(X, y)

LogisticRegression(C=0.75, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:

cross_val_score(logreg, X, y)

array([ 0.14285714,  0.4       ,  0.25      ])

In [54]:
y_pred = logreg.predict(X)
print(classification_report(y, y_pred))
print pd.DataFrame(confusion_matrix(y, y_pred), index=['is_super_low','is_low', 'is_high', 'is_super_high'],
                   columns=['predict_super_low','predict_low', 'predict_high', 'predict_super_high'])

             precision    recall  f1-score   support

       high       0.74      0.52      0.61        27
        low       0.65      0.57      0.61        30
 super_high       0.60      0.76      0.68        34
  super_low       0.59      0.65      0.62        31

avg / total       0.64      0.63      0.63       122

               predict_super_low  predict_low  predict_high  \
is_super_low                  14            3             9   
is_low                         1           17             5   
is_high                        1            1            26   
is_super_high                  3            5             3   

               predict_super_high  
is_super_low                    1  
is_low                          7  
is_high                         6  
is_super_high                  20  


In [36]:
print 'coefficients:\n', logreg.coef_

coefficients:
[[-0.04020536  0.          0.          0.          0.          0.27663998
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.         -0.60747902
   0.          0.          0.          0.2805541   0.          0.          0.
   0.          0.          0.         -0.55435937  0.        ]
 [-0.00162144  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.2478232  -0.61979946  0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.         -0.12084119  0.          0.          0.
   0.         -0.01228179  0.          0.          0.          0.        

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [56]:
## YOUR CODE HERE
logreg = LogisticRegression(C=0.75, penalty='l1', solver='liblinear')
logreg.fit(X, y)
y_pred = logreg.predict(X)
print 'coefficients:', logreg.coef_
print(classification_report(y, y_pred))
print pd.DataFrame(confusion_matrix(y, y_pred), index=['is_super_low','is_low', 'is_high', 'is_super_high'],
                   columns=['predict_super_low','predict_low', 'predict_high', 'predict_super_high'])

coefficients: [[-0.9152989   0.          0.          0.          0.          0.27639268
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.         -0.60766218
   0.          0.          0.          0.28077055  0.          0.          0.
   0.          0.          0.         -0.55428494  0.        ]
 [-0.2143658   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.24764294 -0.62052689  0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.         -0.12118987  0.          0.          0.
   0.         -0.01237522  0.          0.          0.          0.        

In [58]:
## YOUR CODE HERE
logreg = LogisticRegression(C=0.75, penalty='l2', solver='liblinear')
logreg.fit(X, y)
y_pred = logreg.predict(X)
print 'coeficients:', logreg.coef_
print(classification_report(y, y_pred))
print pd.DataFrame(confusion_matrix(y, y_pred), index=['is_super_low','is_low', 'is_high', 'is_super_high'],
                   columns=['predict_super_low','predict_low', 'predict_high', 'predict_super_high'])

coeficients: [[-0.62994219 -0.35109732  0.13947565  0.06304323 -0.16864765  0.75383302
  -0.08324344  0.3658979  -0.14748428 -0.1909422   0.3220382  -0.21384093
  -0.12368925  0.04871492 -0.14748428 -0.13376692 -0.26755232 -0.07273658
   0.60116616  0.50944915  0.32661123  0.23175421  0.03975777 -0.45892723
   0.05106454 -0.14748428 -0.29126246 -0.16864765 -0.16864765 -0.09288879
   0.17703633 -0.09102884 -0.92526519 -0.16864765  0.4284049  -0.08324344
   0.54467301  0.37658279 -0.1338886  -0.23847411  0.44693811 -0.25829818
   0.50854656 -0.60369504  0.19102611]
 [-0.41079635  0.38367034 -0.47318127  0.58972929  0.48176254 -0.34698224
  -0.15700328 -0.06678283  0.44480895 -0.18786009  0.26625689  0.26196841
   0.42440792 -0.51319001 -0.19870702 -0.18436676  0.76952124 -0.6854523
  -0.15700328 -0.19870702 -0.35382175  0.22652417 -0.08020892 -0.05854036
  -0.06807394 -0.19870702  0.14226856  0.48176254  0.48176254 -0.17003089
  -0.4395496  -0.42075682  0.25688477 -0.16879627 -0.14569364

In [41]:
df.head()

,city,job,salary,company,how high,is_high,manager,senior,research,machine,...,data,associate,llc,bank,education,university,ibm,amazon,court,predicted
13,New York,Data Analyst - Cardiology,33650.0,Columbia University,super_low,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,super_high
16,New York,Associate Research Scientist,120000.0,Columbia University,super_high,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,super_high
18,New York,Research and Brand Strategy Apprentice,147500.0,Insight Strategy Group,super_high,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,super_high
22,New York,City Research Scientist II,77975.0,City of New York,low,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,super_high
23,New York,Business Intelligence Engineer,150000.0,Amazon Corporate LLC,super_high,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,super_high


#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

In [59]:
df['predicted'] = y_pred
highest = df.ix[df['predicted'] == 'super_high'][['job','salary']]
print '10 highest paying jobs', '\n', highest.sort_values('salary', ascending=False)[:10]
df.to_csv('Predicted Salaries')

10 highest paying jobs 
                                                   job    salary
64                        Cognitive Software Developer  200000.0
41                                    Research Analyst  200000.0
88   Quantitative Analyst (Fundamental Quant Equiti...  200000.0
66                 Research Scientist: Control Systems  200000.0
117            Associate Quantitative Research Analyst  180000.0
273                        Visual Interaction Designer  180000.0
259                      Analyst - Sales Ops & Finance  180000.0
200                Indoor Environmental Data Scientist  176800.0
31                           Research Engineer, Senior  175000.0
256             Software Engineer (Performance & Data)  165000.0


### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [ ]:
## YOUR CODE HERE